In [2]:
import pandas as pd
from pytrends.request import TrendReq
import os
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from nltk.collocations import TrigramAssocMeasures, TrigramCollocationFinder
from nltk.collocations import QuadgramAssocMeasures, QuadgramCollocationFinder
import statistics
import collections
from dotenv import load_dotenv

In [3]:
# Cargar las variables de entorno desde el archivo .env
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [4]:
# Initialize NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabriel.hernan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gabriel.hernan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabriel.hernan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Initialize pytrends request
pytrends = TrendReq(hl='es-MX', tz=360)

# List of keywords we want to analyze
keywords_list = ["ia", "search engine optimization", "analytics", "web development", "technology" ]

# Initialize an empty DataFrame to store the results
all_rising_topics_df = pd.DataFrame()

# Iterate over each keyword in the list
for keyword in keywords_list:
    # Build payload for the keyword
    pytrends.build_payload([keyword], cat=0, timeframe="today 3-m", geo="MX")
    
    # Get related topics for the keyword
    related_topics = pytrends.related_topics()
    
    # Extract rising topics for the keyword, if available
    if related_topics and keyword in related_topics and 'rising' in related_topics[keyword]:
        rising_df = related_topics[keyword]['rising'].copy()  # Make a copy of the rising topics DataFrame
        # Normalize the DataFrame by dropping unnecessary columns, if desired
        rising_df = rising_df.drop(columns=['link', 'topic_mid'])
        # Add a 'keyword' column to indicate which keyword the topics belong to
        rising_df['keyword'] = keyword
        # Append the rising topics DataFrame to the main all_rising_topics_df
        all_rising_topics_df = pd.concat([all_rising_topics_df, rising_df], ignore_index=True)

# Display the final concatenated DataFrame of rising topics for all keywords
print(all_rising_topics_df)

      value formattedValue                                        topic_title  \
0     78450        Aumento                        Generador de imágenes de IA   
1     22000        Aumento                                    Marcela Mistral   
2     14650        Aumento                       Leonardo Interactive Pty Ltd   
3       200         +200 %                                               NSFW   
4        50          +50 %                                      Renderización   
5        40          +40 %                                            Censura   
6   2043300        Aumento                                          Marketing   
7     58600        Aumento                                     Tasa de rebote   
8     42800        Aumento                                          Propiedad   
9     35250        Aumento                          Cualificación profesional   
10    34450        Aumento                                Universidad Harvard   
11    31800        Aumento  

In [5]:
all_rising_topics_df.to_csv('rising_topics.csv', index=False, encoding='utf-8-sig')

In [11]:
# Se asume que all_rising_topics_df es un DataFrame que contiene las tendencias y que ya está importado o disponible en tu código.

country = "ES"

# Función modificada para recibir una lista de consultas en lugar de una sola

def google_custom_search_df(query, country):
    API_CUSTOM_SEARCH_KEY = os.getenv('API_CUSTOM_SEARCH_KEY')
    API_CUSTOM_SEARCH_ID = os.getenv('API_CUSTOM_SEARCH_ID')
    
    # Crear un DataFrame vacío para almacenar todos los resultados concatenados
    all_search_results_df = pd.DataFrame()

    # Iterar sobre la lista de consultas
    for q in query:
        # URL request con la consulta actual
        url = f"https://www.googleapis.com/customsearch/v1?key={API_CUSTOM_SEARCH_KEY}&cx={API_CUSTOM_SEARCH_ID}&q={q}&start=1&gl={country}"
        data = requests.get(url).json()

        # Extraer lo que contiene items:
        items = data.get("items")

        # Si hay resultados, crear un DataFrame y añadirlo al DataFrame principal
        if items:
            df = pd.DataFrame(items, columns=['title', 'link'])
            df['query'] = q  # Añadir columna con la consulta para referencia
            all_search_results_df = pd.concat([all_search_results_df, df], ignore_index=True)
            
    # Después de concatenar los resultados de búsqueda, imprime para verificar la estructura
    print("all_search_results_df estructura:", all_search_results_df.columns)
    print(all_search_results_df.head())  # Muestra las primeras filas para verificar

    return all_search_results_df



In [ ]:
# Función de scraping adaptada para manejar errores y vacíos
# Utiliza tu código de scraping existente
def scrape_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        if not article.text:
            # Si Newspaper no pudo obtener el texto, intenta con BeautifulSoup
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            
            # Busca contenido en etiquetas <p> o <div>
            article_text = ' '.join([p.get_text() for p in soup.find_all(['p', 'div'])])
            
            return article_text
        return article.text
    except Exception as e:
        print(f"Error al scrapear el artículo: {str(e)}")
        return ""

# Función para aplicar scraping a una lista de artículos y crear una columna de texto extraído
def scrape_articles_in_dataframe(df):
    scraped_texts = []  # Aquí almacenaremos el texto de los artículos

    for url in df['link']:
        scraped_text = scrape_article(url)
        scraped_texts.append(scraped_text)

    # Agregamos los textos como una nueva columna en el DataFrame
    df['scraped_text'] = scraped_texts

    return df

# Utilizando el DataFrame de tendencias, extrae los títulos de tendencia como consultas
trend_queries = all_rising_topics_df['topic_title'].tolist()

# Usa la función modificada con la lista de consultas
all_search_results_df = google_custom_search_df(trend_queries, country)

# Realiza el scraping de los contenidos de los resultados de búsqueda y añade texto raspado a DataFrame
all_search_results_df = scrape_articles_in_dataframe(all_search_results_df)

# A este punto, all_search_results_df contendrá los resultados de búsqueda y el texto de cada artículo raspado, con una columna adicional 'query' que indica la consulta original.

In [13]:
#--------------- GOOGLE CUSTOM SEARCH: nos conectamos a la API de Google a través de generar un buscador global(tambien puede ser para un site en especifico)

query = "marketing digital"
country = "ES"
leng = "ES"


def google_custom_search_df(query, country="ES"):
    API_CUSTOM_SEARCH_KEY = os.getenv('API_CUSTOM_SEARCH_KEY')
    API_CUSTOM_SEARCH_ID = os.getenv('API_CUSTOM_SEARCH_ID')

    # URL request
    url = f"https://www.googleapis.com/customsearch/v1?key={API_CUSTOM_SEARCH_KEY}&cx={API_CUSTOM_SEARCH_ID}&q={query}&start=1&gl={country}"
    data = requests.get(url).json()

    # Extraer lo que contiene items:
    items = data.get("items")

    # Crear un DataFrame con los resultados de la búsqueda
    df = pd.DataFrame(items, columns=['title', 'link'])

    return df

df = google_custom_search_df(query, country)


#-------------------SCRAPING del texto de los articulos de las 10 primeras posiciones de Google SERP

#Extraer el texto de los articulos con la libreria newspapper y, en su defecto, con beautiful soup

def scrape_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        if not article.text:
            # Si Newspaper no pudo obtener el texto, intenta con BeautifulSoup
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            
            # Busca contenido en etiquetas <p> o <div>
            article_text = ' '.join([p.get_text() for p in soup.find_all(['p', 'div'])])
            
            return article_text
        return article.text
    except Exception as e:
        print(f"Error al scrapear el artículo: {str(e)}")
        return ""

def scrape_articles_in_dataframe(df):
    scraped_texts = []  # Aquí almacenaremos el texto de los artículos

    for url in df['link']:
        scraped_text = scrape_article(url)
        scraped_texts.append(scraped_text)

    # Agregamos los textos como una nueva columna en el DataFrame
    df['scraped_text'] = scraped_texts

    return df

# Llama a la función scraping de los artículos
df = scrape_articles_in_dataframe(df)
